#### 7. 时间序列数据处理

##### 一、时序数据中的基本对象

时间序列的概念在日常生活中十分常见，但对于一个具体的时序事件而言，可以从多个时间对象的角度来描述。例如2020年9月7日周一早上8点整需要到教室上课，这个课会在当天早上10点结束，其中包含了哪些时间概念？

* 第一，会出现时间戳（Date times）的概念，即'2020-9-7 08:00:00'和'2020-9-7 10:00:00'这两个时间点分别代表了上课和下课的时刻，在`pandas`中称为`Timestamp`。同时，一系列的时间戳可以组成`DatetimeIndex`，而将它放到`Series`中后，`Series`的类型就变为了`datetime64[ns]`，如果有涉及时区则为`datetime64[ns, tz]`，其中tz是timezone的简写。

* 第二，会出现时间差（Time deltas）的概念，即上课需要的时间，两个`Timestamp`做差就得到了时间差，pandas中利用`Timedelta`来表示。类似的，一系列的时间差就组成了`TimedeltaIndex`， 而将它放到`Series`中后，`Series`的类型就变为了`timedelta64[ns]`。

* 第三，会出现时间段（Time spans）的概念，即在8点到10点这个区间都会持续地在上课，在`pandas`利用`Period`来表示。类似的，一系列的时间段就组成了`PeriodIndex`， 而将它放到`Series`中后，`Series`的类型就变为了`Period`。

* 第四，会出现日期偏置（Date offsets）的概念，假设你只知道9月的第一个周一早上8点要去上课，但不知道具体的日期，那么就需要一个类型来处理此类需求。再例如，想要知道2020年9月7日后的第30个工作日是哪一天，那么时间差就解决不了你的问题，从而`pandas`中的`DateOffset`就出现了。同时，`pandas`中没有为一列时间偏置专门设计存储类型，理由也很简单，因为需求比较奇怪，一般来说我们只需要对一批时间特征做一个统一的特殊日期偏置。

通过这个简单的例子，就能够容易地总结出官方文档中的这个[表格](https://pandas.pydata.org/docs/user_guide/timeseries.html#overview)：

|概念 |                          单元素类型              |    数组类型         |                pandas数据类型|
|:---------|:----------|:-----------|:------------|
|Date times           |           `Timestamp`       |       `DatetimeIndex`  |   `datetime64[ns]`|
|Time deltas          |           `Timedelta`        |      `TimedeltaIndex` |  `timedelta64[ns]`|
|Time spans            |          `Period`           |      `PeriodIndex`   |    `period[freq]`|
|Date offsets          |          `DateOffset`         |    `None`          |    `None`|

由于时间段对象`Period/PeriodIndex`的使用频率并不高，因此将不进行讲解，而只涉及时间戳序列、时间差序列和日期偏置的相关内容。


In [2]:
import pandas as pd
import numpy as np

##### 二、时间戳

时间戳是应用很广泛的时间类型，也是日常数据处理与分析常见的时间类型。主要介绍时间戳的构造、属性及切片索引。

##### 1) 时间戳的构造与属性

单个时间戳的生成利用`pd.Timestamp`实现，一般而言的常见日期格式都能被成功地转换：

In [3]:
ts=pd.Timestamp('2022/10/18')
ts

Timestamp('2022-10-18 00:00:00')

In [44]:
ts = pd.Timestamp('2022-10-18 08:10:30')
ts

Timestamp('2022-10-18 08:10:30')

对于一个时间戳来说，它是由若干个基本时间单位确定，包括年、月、日、小时、分、秒等，可以通过year、month、day、hour、min、second来获取

In [48]:
ts.year
ts.month
ts.day
ts.hour
# ts.min
# ts.second


8

##### 2. Datetime序列的生成

一组时间戳可以组成时间序列，可以用`to_datetime`和`date_range`来生成。其中，`to_datetime`能够把一列时间戳格式的对象转换成为`datetime64[ns]`类型的时间序列：

In [51]:
# pd.Series(pd.to_datetime(['2020-1-1', '2020-1-3', '2020-1-6']))
df = pd.read_csv('data/learn_pandas.csv')
s = pd.to_datetime(df.Test_Date)
s.head()

0   2019-10-05
1   2019-09-04
2   2019-09-12
3   2020-01-03
4   2019-11-06
Name: Test_Date, dtype: datetime64[ns]

也可以在文件读取时，直接通过参数para_dates将制定的某一列或多列解析为datetime64[ns]类型。

In [10]:
df=pd.read_csv('data/learn_pandas.csv',parse_dates=['Test_Date'])
s=df.Test_Date
s.head()

0   2019-10-05
1   2019-09-04
2   2019-09-12
3   2020-01-03
4   2019-11-06
Name: Test_Date, dtype: datetime64[ns]

在极少数情况，时间戳的格式不满足转换时，可以强制使用`format`进行匹配。

Y、m、d、H,M,S分别代表年、月、日、小时、分、秒。

In [56]:
temp = pd.to_datetime(['2020&1&1','2020&1&3'],format='%Y&%m&%d')
temp

DatetimeIndex(['2020-01-01', '2020-01-03'], dtype='datetime64[ns]', freq=None)

注意上面由于传入的是列表，而非`pandas`内部的`Series`，因此返回的是`DatetimeIndex`，如果想要转为`datetime64[ns]`的序列，需要显式用`Series`转化：

In [53]:
pd.Series(temp).head()

0   2020-01-01
1   2020-01-03
dtype: datetime64[ns]

##### 随堂练习


请将如下的格式的时间戳列表转化为正确格式的时间序列Series

（1）['0901 2021','0902 2021', '0903 2021']

(2)['2021-9-1 8-35-50','2021-9-2 9-25-45']


In [58]:








ds1=['0901 2021','0902 2021', '0903 2021']

ds2=['2021-9-1 8-35-50','2021-9-2 9-25-45']

temp=pd.to_datetime(ds1,format='%m%d %Y')
pd.Series(temp)

temp=pd.to_datetime(ds2,format='%Y-%m-%d %H-%M-%S')
pd.Series(temp)

0   2021-09-01 08:35:50
1   2021-09-02 09:25:45
dtype: datetime64[ns]

另外，还存在一种把表的多列时间属性拼接转为时间序列的`to_datetime`操作，此时的列名必须和以下给定的时间关键词列名（year、month、day、hour、minute、second）一致：

In [60]:
df_date_cols = pd.DataFrame({'year': [2020, 2020],
                             'month': [1, 1],
                             'day': [1, 2],
                             'hour': [10, 20],
                             'minute': [30, 50],
                             'second': [20, 40]})

df_date_cols
pd.to_datetime(df_date_cols)

0   2020-01-01 10:30:20
1   2020-01-02 20:50:40
dtype: datetime64[ns]

`date_range`是一种生成连续间隔时间的一种方法，其重要的参数为`start, end, freq, periods`，它们分别表示开始时间，结束时间，时间间隔，时间戳个数。其中，四个中的三个参数决定了，那么剩下的一个就随之确定了。这里要注意，开始或结束日期如果作为端点则它会被包含：


参数freq：数据格式是string或dateOffset,默认值是‘D’，表示以自然日为单位。

常用的时间序列频率包括：
* B       工作日频率
* D       日历日频率
* W       每周频率
* BH      工作小时级频率
* H       小时级频率
* T,min   分钟级频率
* S       秒级频率
* M       月结束频率，如'2018-11-30', '2018-12-31'
* SM      半月结束频率（15日和月末）
* BM      工作月结束频率

In [63]:
pd.date_range('2020-1-1','2020-1-21', freq='10D')

DatetimeIndex(['2020-01-01', '2020-01-11', '2020-01-21'], dtype='datetime64[ns]', freq='10D')

In [25]:
pd.date_range('2020-1-1', '2020-2-28', periods=6)

DatetimeIndex(['2020-01-01 00:00:00', '2020-01-12 14:24:00',
               '2020-01-24 04:48:00', '2020-02-04 19:12:00',
               '2020-02-16 09:36:00', '2020-02-28 00:00:00'],
              dtype='datetime64[ns]', freq=None)

##### 3). dt对象

pandas在时间序列上定义了`dt`对象来完成许多时间序列的相关操作。这里对于`datetime64[ns]`类型而言，可以大致分为三类操作：取出时间相关的属性、判断时间戳是否满足条件、取整操作。

* 第一类操作的常用属性包括：`date, time, year, month, day, hour, minute, second, microsecond, nanosecond, dayofweek, dayofyear, weekofyear, daysinmonth, quarter`，其中`daysinmonth, quarter`分别表示该月一共有几天和季度。

In [71]:
s = pd.Series(pd.date_range('2022-10-1','2022-10-31', freq='10D'))
s.dt.date
s.dt.time
s.dt.day
s.dt.daysinmonth #时间戳所在的月份共有几天
s.dt.dayofweek #这一周的第几天

0    5
1    1
2    4
3    0
dtype: int64

* 第二类操作主要用于测试时间戳是否为年、季、月的第一天或最后一天。

In [75]:
# s.dt.is_month_end # 还可选 is_quarter/month_start
s.dt.is_quarter_end

0    False
1    False
2    False
3    False
dtype: bool

##### 4) 切片

一般而言，时间戳序列作为索引使用。如果想要选出某个子时间戳序列，第一类方法是利用`dt`对象和布尔条件联合使用，另一种方式是利用切片，后者常用于连续时间戳。

* 第一种方法，使用dt对象和布尔条件

In [79]:

s=pd.Series(np.random.randint(2,size=365),index=pd.date_range('2022-01-01','2022-12-31'))
idx=pd.Series(s.index).dt #获得dt对象
s.head(10)

pd.Series(s.index).dt

In [83]:
#取出每月的第一天和最后一天
# idx.is_month_end
condition=idx.is_month_start | idx.is_month_end
condition
s[condition.values].head()

2022-01-01    0
2022-01-31    0
2022-02-01    1
2022-02-28    1
2022-03-01    0
dtype: int64

In [86]:
# 取出所有的双休日
idx.dayofweek
condition=idx.dayofweek.isin([5,6])
condition
s[condition.values].head()

2022-01-01    0
2022-01-02    0
2022-01-08    1
2022-01-09    1
2022-01-15    0
dtype: int64

* 第二类方法：切片

In [87]:
s['2022-01-01'] #单值索引

0

In [89]:
s['2022-05-15':'2022-07-05']

2022-05-15    1
2022-05-16    0
2022-05-17    1
2022-05-18    0
2022-05-19    0
2022-05-20    0
2022-05-21    0
2022-05-22    1
2022-05-23    1
2022-05-24    1
2022-05-25    1
2022-05-26    0
2022-05-27    0
2022-05-28    0
2022-05-29    1
2022-05-30    1
2022-05-31    0
2022-06-01    1
2022-06-02    1
2022-06-03    1
2022-06-04    0
2022-06-05    1
2022-06-06    1
2022-06-07    1
2022-06-08    0
2022-06-09    0
2022-06-10    0
2022-06-11    1
2022-06-12    1
2022-06-13    0
2022-06-14    0
2022-06-15    0
2022-06-16    1
2022-06-17    1
2022-06-18    1
2022-06-19    1
2022-06-20    0
2022-06-21    0
2022-06-22    1
2022-06-23    1
2022-06-24    0
2022-06-25    1
2022-06-26    1
2022-06-27    1
2022-06-28    0
2022-06-29    1
2022-06-30    1
2022-07-01    0
2022-07-02    1
2022-07-03    1
2022-07-04    0
2022-07-05    1
Freq: D, dtype: int64

##### 练习：

在上述构造的Series 中，按照如下要求选出子序列

* 选出每月前10天中的工作日
* 选出每月的1日、11日和21日以及每月的最后5天。


In [113]:






# 选出每月前10天中的工作日
s_tmp=s[(idx.day<10).values] #获得每月的前10天，得到子序列
s_idx=pd.Series(s_tmp.index).dt #获得子序列的时间序列
s_tmp[(s_idx.dayofweek<5).values].head(10) #从子序列中判断是否为工作日。


2021-01-01    0
2021-01-04    0
2021-01-05    0
2021-01-06    1
2021-01-07    0
2021-01-08    1
2021-02-01    0
2021-02-02    0
2021-02-03    1
2021-02-04    1
dtype: int64

In [126]:
#选出每月的1日、11日和21日，以及每月的最后5天
condition=((idx.day % 10==1) | (idx.daysinmonth-idx.day<=4)).values
condition

s[condition]


2021-01-01    0
2021-01-11    0
2021-01-21    0
2021-01-27    1
2021-01-28    1
             ..
2021-12-27    0
2021-12-28    0
2021-12-29    1
2021-12-30    0
2021-12-31    1
Length: 96, dtype: int64

#### 三、时间差

时间差是中被广泛研究的对象，例如，用户点击广告后的存留时间、学生的长跑体测成绩、电子产品的预期寿命等。


##### 1. 时间差序列生成

时间差可以理解为两个时间戳的差，这里也可以通过`pd.Timedelta`来构造：

In [49]:
pd.Timestamp('20210102 08:00:00')-pd.Timestamp('20210101 07:35:00')


Timedelta('1 days 00:25:00')

In [50]:
pd.Timedelta(days=1, minutes=25) #1天25分钟

Timedelta('1 days 00:25:00')

生成时间差序列的主要方式是`pd.to_timedelta`，其类型为`timedelta64[ns]`：

In [52]:
df.Time_Record

0      0:04:34
1      0:04:20
2      0:05:22
3      0:04:08
4      0:05:22
        ...   
195    0:04:31
196    0:04:03
197    0:04:48
198    0:04:58
199    0:05:05
Name: Time_Record, Length: 200, dtype: object

In [51]:
s=pd.to_timedelta(df.Time_Record)
s

0     0 days 00:04:34
1     0 days 00:04:20
2     0 days 00:05:22
3     0 days 00:04:08
4     0 days 00:05:22
            ...      
195   0 days 00:04:31
196   0 days 00:04:03
197   0 days 00:04:48
198   0 days 00:04:58
199   0 days 00:05:05
Name: Time_Record, Length: 200, dtype: timedelta64[ns]

##### 2. 时间差序列运算

时间差对象支持3类运算：

* 与标量的乘法运算
* 与时间戳的加减法运算
* 与时间差的加减法和除法运算

In [53]:
td1=pd.Timedelta(days=1)
td2=pd.Timedelta(days=3)
ts=pd.Timestamp('20210101')


In [54]:
td1*2 #标量计算

Timedelta('2 days 00:00:00')

In [59]:
td2/td1 #时间差之间的加减法运算

3.0

In [57]:
td2-td1

Timedelta('2 days 00:00:00')

In [58]:
td2+ts # 与时间戳的加减法

Timestamp('2021-01-04 00:00:00')

##### 练习
##### 水果销量数据集

现有一份2019年每日水果销量记录表：

In [61]:
df = pd.read_csv('data/fruit.csv')
df.head(3)

,Date,Fruit,Sale
0,2019-04-18,Peach,15
1,2019-12-29,Peach,15
2,2019-06-05,Peach,19


1. 统计如下指标：
* 每月最后一天的生梨销量总和
* 每月最后一个工作日的生梨销量总和